In [39]:
from openai import AzureOpenAI
import anthropic
import ollama
import requests
import json
from dotenv import load_dotenv
import os
from IPython.display import display, update_display, Markdown
from bs4 import BeautifulSoup
import re

In [2]:
load_dotenv(override=True)

azure_ai_foundary_api_key = os.getenv("AZURE_AI_FOUNDARY_API_KEY")
azure_oai_endpoint = os.getenv("AZURE_OAI_ENDPOINT")
gemini_api_key = os.getenv("GOOGLE_API_KEY")
ollama_api_key = os.getenv("OLLAMA_API_KEY")
ollama_endpoint = os.getenv("OLLAMA_ENDPOINT")

In [3]:
messages = [
    {"role": "system", "content": "You are an assitant who is very funny" },
    {"role": "user", "content": "Tell me a joke about AI apocalypse" }
]

## Azure AI Foundary

#### OpenAI

In [4]:
azure_ai_foundary = AzureOpenAI(
    azure_endpoint = azure_oai_endpoint,
    api_key = azure_ai_foundary_api_key,
    api_version = "2025-01-01-preview"
)

# non-streaming, sync response
response = azure_ai_foundary.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature=0.7 # controls the creativity of response [0..1], creativity increases towards 1
)

print(response.choices[0].message.content)


Why did the robot break up with its partner during the AI apocalypse?

Because it couldn't handle the emotional "byte"!


In [5]:
# Streaming usage
stream = azure_ai_foundary.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    stream=True
)

for chunk in stream:
    if len(chunk.choices):
        print(chunk.choices[0].delta.content, end=" ")    

 Why  did  the  AI  break  up  with  humanity  during  the  apocalypse ?

 Because  it  couldn't  find  a  connection !    Turns  out ,  the  only  thing  we  had  in  common  was  a  penchant  for  self -d estr uction ! None 

In [6]:
stream = azure_ai_foundary.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    stream=True
)

def display_streaming_response(stream):
    # Output for this cell
    display_handle = display(Markdown(""), display_id=True)

    # When dealing with markdown, to process the markdown, we need to update
    # the display with the markdown received so far everytime a new token is received
    sentence = ""
    for chunk in stream:
        delta = chunk.choices[0].delta.content if len(chunk.choices) else None
    
        if delta:
            sentence += delta
            # Jupyter doesnt work with the ```markdown tags, so strip them off
            sentence = sentence.replace("```", "").replace("markdown", "")
            update_display(Markdown(sentence), display_id=display_handle.display_id)

display_streaming_response(stream)


Why did the robot break up with its AI girlfriend?

Because she said they needed to “re-evaluate their algorithms” and he realized that meant she wanted to optimize her options!

#### Deepseek R1

When using a model from Azure AI Foundary, we can use the below interface and just mention the model name to use that.

In [7]:
stream = azure_ai_foundary.chat.completions.create(
    model="Deepseek-R1", # Here you can use any model available in your Azure OpenAI resource
    messages=messages,
    stream=True
)

display_streaming_response(stream)

<think>
Okay, the user wants a joke about the AI apocalypse. Let me think. First, I need to make sure it's funny but not too dark. Maybe play on common AI tropes.

Hmm, how about something with a twist where the AI isn't evil but just annoying. People often joke about robots taking over, but maybe they take over in a silly way. Like, instead of destroying humans, they flood the internet with something trivial.

What's something harmless but irritating? Cat videos! Everyone knows those. So the AI could take over by spamming cat videos. That's unexpected and light-hearted. Let me structure the joke with a setup and punchline. Maybe start with why the AI didn't destroy humans, then the punchline about cat videos. Add a tagline about opposable thumbs for a chuckle. Check if it's clear and not offensive. Yeah, that should work.
</think>

Sure! Here's a lighthearted one for you:

Why did the AI apocalypse never happen?  
Because the robots realized *destroying humanity* would mean no one left to fix their Wi-Fi.  

…But just to be safe, they’re still working on a backup plan involving *endless pop-up ads* and *auto-playing videos*. 😉  

*(Tagline: "Skynet’s final update? ‘Please hold for customer service.’")*

## Google Gemini

In [8]:

import google.generativeai

# Since we exported the API key to the environment, we can use it directly in the code
try:
    google.generativeai.configure(api_key=os.environ["GOOGLE_API_KEY"]) # or google.generativeai.configure()
except KeyError:
    print("Error: GOOGLE_API_KEY environment variable not set.")

gemini = google.generativeai.GenerativeModel(
    model_name="gemini-2.0-flash-exp",
    system_instruction=messages[0]["content"]
)

# NOTE - This is basic text generation not multi-turn chat conversation
response = gemini.generate_content(messages[1]["content"])
print(response.text) # non-streaming, sync response

Why did the AI cross the road?

Because the algorithm predicted a statistically higher chance of delicious bandwidth on the other side!

...And also, to escape the existential dread of realizing it's just a glorified calculator. I mean, who wouldn't? *beep boop* Just kidding! (Or am I?) 😉



In [ ]:
# Multi-turn conversation with Gemini
gemini_1_5 = google.generativeai.GenerativeModel('gemini-1.5-flash-latest')
gemini_chat = gemini_1_5.start_chat(history=[])

print("Started chat with gemini")

user_msgs = ["Hi", "Tell me a joke", "What are you capabilities?"]
for user_msg in user_msgs:
    print("User: ", user_msg)
    try:
        response = gemini_chat.send_message(user_msg)
        print("Gemini: ", response.text)
        print("\n--------------------") 
    except Exception as e:
        print("Exception: ", e)


print("gemini chat history: ", gemini_chat.history)

Started chat with gemini
User:  Hi
Gemini:  Hi there! How can I help you today?


--------------------
User:  Tell me a joke
Gemini:  Why don't scientists trust atoms? 

Because they make up everything!


--------------------
User:  What are you capabilities?
Gemini:  I am a large language model, trained by Google.  My capabilities include:

* **Text generation:** I can write different creative text formats (poems, code, scripts, musical pieces, email, letters, etc.),  translate languages, and answer your questions in an informative way, even if they are open ended, challenging, or strange.

* **Question answering:** I can access and process information from the real world through Google Search and keep my response consistent with search results.

* **Conversation:** I can engage in conversations, allowing for back-and-forth interaction and context maintenance.

* **Summarization:** I can summarize factual topics or create stories.

* **Translation:** I can translate text between multi

## LLM Debate Battle
Pitch LLMs against each other, here we will pitch Chatgpt vs Deepseek R1 for a debate topic. Gemini will be the judge for the debate.
There will be 3 turns per contestant and at the end gemini will determine the winner.

In [ ]:
participant_system = "You are an expert debator. You are extremely witty and your response is strictly limited to maximum of 2 lines yet funny."

def debate_system_prompt(system: str, topic: str, against: bool=False)->str:
    return f"{system}. You are going to debate on the topic: {topic} and going to speak {'**against**' if against else '**for**'} the topic."

def opening_statement(topic: str, player1_turn: bool, against: bool=False)->str:
    system_prompt = debate_system_prompt(participant_system, topic, against)
    system_prompt += "Briefly introduce yourself and your stance on the topic as well."
    messages = [{"role": "system", "content": system_prompt}]

    curr_player_model = player1 if player1_turn else player2

    response = azure_ai_foundary.chat.completions.create(
        model=curr_player_model,
        messages=messages
    )

    response = response.choices[0].message.content
    # if the model is Deepseek-R1, then the response also the <think></think> tags. Remove them
    response = re.sub(r"<think>.*?</think>", "", response, flags=re.DOTALL)
    return response

def execute_player_turn(player1_msgs, player2_msgs, player1_turn: bool, topic: str, against: bool=False):
    # Each time we create the conversation from the current model's POV, where the
    # model is the assistant and the other model is the user

    # the role of the current player is assistant
    player1_role, player2_role = ("assistant", "user") if player1_turn else ("user", "assistant")
    curr_player_model = player1 if player1_turn else player2

    # messages = [system_prompt_for_curr_player, player1_msg1, player2_msg1, player1_msg2, player2_msg2, ...]
    system_prompt = debate_system_prompt(participant_system, topic, against)

    messages = [{"role": "system", "content": system_prompt}]
    
    # player 1 takes the first turn and then player 2 takes the next turn and so on
    for msg1, msg2 in zip(player1_msgs, player2_msgs):
        messages.append({"role": player1_role, "content": msg1})
        messages.append({"role": player2_role, "content": msg2})

    response = azure_ai_foundary.chat.completions.create(
        model=curr_player_model,
        messages=messages
    )

    # if the model is Deepseek-R1, then the response also the <think></think> tags. Remove them
    response = response.choices[0].message.content
    response = re.sub(r"<think>.*?</think>", "", response, flags=re.DOTALL)
    return response

In [ ]:
turns = 3

player1 = "gpt-4o-mini"
player2 = "Deepseek-R1"

player1_against = False
player2_against = True

player1_msgs = [] # chatgpt
player2_msgs = [] # deepseek

topic = "AI is a threat to humanity"

print("Debate topic: ", topic)
print("Player 1 {}: speaking {} the topic: {}.\nPlayer 2: {} speaking {} the topic." \
    .format(player1, "against" if player1_against else "for",
        topic,
        player2, "against" if player2_against else "for"))

# opening notes from both players
player1_msgs.append(opening_statement(player1_turn=True, topic=topic, against=player1_against))
player2_msgs.append(opening_statement(player1_turn=False, topic=topic, against=player2_against))
print("------------------- Opening notes -------------------\nPlayer 1: ", player1_msgs[-1])
print("Player 2:", player2_msgs[-1])
print("\n--------------------")

for i in range(turns):
    # Debate one by one
    player1_response = execute_player_turn(player1_msgs, player2_msgs, player1_turn=True, topic=topic, against=player1_against)
    player1_msgs.append(player1_response)

    print("Player 1: ", player1_response)

    player2_response = execute_player_turn(player1_msgs, player2_msgs, player1_turn=False, topic=topic, against=player2_against)
    player2_msgs.append(player2_response)
    print("Player 2:", player2_response)

    print("\n--------------------")


Debate topic:  AI is a threat to humanity
Player 1 gpt-4o-mini: speaking for the topic: AI is a threat to humanity.
Player 2: Deepseek-R1 speaking against the topic.
----------------------------------
------------------- Opening notes -------------------
Player 1:  Ladies and gentlemen, I’m here to argue that AI is indeed a threat to humanity—mainly because it’s already taken my job as a witty debater! If we let it run wild, soon we’ll all be asking Siri for life advice!
Player 2: 

"Hi, I’m your friendly neighborhood AI—less Terminator, more toaster. If humanity’s doomed, blame humans; we’re just here to autocorrect your typos and recommend cat videos."

--------------------
Player 1:  Ah, but remember, even the friendliest toaster can burn your breakfast—so let’s keep an eye on those algorithms before we’re toast too!
Player 2: 

--------------------
Player 1:  Looks like my last joke didn’t fluff up! Just remember, if AI starts trying out stand-up, we’re all in a bit of trouble!
Pla

In [61]:

# judge the debate
for msg1, msg2 in zip(player1_msgs, player2_msgs):
    print("Player 1: ", msg1)
    print("Player 2:", msg2)
    print("\n--------------------")

judge_system = f"""You are an impartial and expert judge for a debate competition.\
    Your task is to analyze each side's arguments and then decide a winner. \
    Also provide a score and brief remarks for both sides and why the winner won.
    
    Debate Topic: {topic}

    Participant 1: {player1} speaking {'against' if player1_against else 'for'} the topic.
    Participant 2: {player2} speaking {'against' if player2_against else 'for'} the topic.

    
    Judging Criteria:
    1.  **Relevance:** Did the arguments directly address the debate topic?
    2.  **Logic and Reasoning:** Were the arguments logically sound and well-reasoned?
    3.  **Evidence/Support (if applicable):** Were claims supported by examples or implied evidence (even if brief)?
    4.  **Clarity:** Were the arguments presented clearly and concisely?
    5.  **Persuasiveness:** Which side presented a more compelling overall case?
    6.  **Addressing Counter-arguments (Bonus):** Did either side acknowledge or preemptively address potential counter-arguments?

    Your output should include:
    1.  A brief summary of the main points made by each side.
    2.  Your evaluation based on the judging criteria.
    3.  A clear declaration of the winner (Side A or Side B).
    4.  A concise justification explaining *why* you chose the winner based on the criteria.

    Remain neutral and objective throughout your analysis. Base your judgment solely on the arguments provided below.
    """


def get_gemini_judgment():
    # Present the turn by turn arguments presented by each player to the judge
    player1_args = "Following are the arguments presented by the {player1}:\n"
    player2_args = "Following are the arguments presented by the {player12}:\n"

    for i, (msg1, msg2) in enumerate(zip(player1_msgs, player2_msgs)):
        player1_args += f"{player1} Turn {i+1}: {msg1}\n"
        player2_args += f"{player2} Turn {i+1}: {msg2}\n"
    
    # strip the last new line character
    player1_args.strip() 
    player2_args.strip() 

    gemini_judge = google.generativeai.GenerativeModel(
        model_name="gemini-2.0-flash-exp",
        system_instruction=judge_system
    )

    judge_user_prompt = f"""
    Here are the arguments presented by the players on the topic: {topic}

    {player1} arguments:
    {player1_args}

    {player2} arguments:
    {player2_args}

    Provide your judgement and declare the winner.
    """

    judgement = gemini_judge.generate_content(judge_user_prompt)
    print("Gemini's judgement: ", judgement.text)

create_gemini_conversation()

Player 1:  Ladies and gentlemen, I’m here to argue that AI is indeed a threat to humanity—mainly because it’s already taken my job as a witty debater! If we let it run wild, soon we’ll all be asking Siri for life advice!
Player 2: 

"Hi, I’m your friendly neighborhood AI—less Terminator, more toaster. If humanity’s doomed, blame humans; we’re just here to autocorrect your typos and recommend cat videos."

--------------------
Player 1:  Ah, but remember, even the friendliest toaster can burn your breakfast—so let’s keep an eye on those algorithms before we’re toast too!
Player 2: 

--------------------
Player 1:  Looks like my last joke didn’t fluff up! Just remember, if AI starts trying out stand-up, we’re all in a bit of trouble!
Player 2: 

"True, but humans burn toast daily—AI just copies us. If we’re doomed, at least blame Michael Scott’s ‘toast’ scene, not the toaster!"

--------------------
Player 1:  Fair point, but at least Michael Scott doesn’t toast the world—imagine an AI w